In [2]:
import urllib.request
import zipfile
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

**Download dan Ekstrak Dataset**

In [52]:
# Download dan ekstrak dataset training
data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
with zipfile.ZipFile('horse-or-human.zip', 'r') as zip_ref:
    zip_ref.extractall('data/horse-or-human')

# Download dan ekstrak dataset validasi
data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
with zipfile.ZipFile('validation-horse-or-human.zip', 'r') as zip_ref:
    zip_ref.extractall('data/validation-horse-or-human')

**Data Preprocessing (ImageDataGenerator)**

In [53]:
TRAINING_DIR = 'data/horse-or-human'
VALIDATION_DIR = 'data/validation-horse-or-human'

train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


**Build Model**

In [76]:
model = tf.keras.models.Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Dropout(0.5),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # DO NOT CHANGE THIS LINE!
])


**Compile dan Train Model**

In [77]:
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

early_stop = EarlyStopping(
    monitor='val_accuracy',  # Monitoring validation accuracy
    patience=10,
    restore_best_weights=True,  # This ensures we keep the best weights
    verbose=1  # To see when early stopping occurs
)

history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 17s 425ms/step - accuracy: 0.5285 - loss: 0.7935 - val_accuracy: 0.5000 - val_loss: 0.9191
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 12s 351ms/step - accuracy: 0.7673 - loss: 0.4697 - val_accuracy: 0.7031 - val_loss: 1.1017
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 12s 352ms/step - accuracy: 0.8650 - loss: 0.3406 - val_accuracy: 0.6484 - val_loss: 2.0487
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 12s 349ms/step - accuracy: 0.9271 - loss: 0.2184 - val_accuracy: 0.8164 - val_loss: 1.2166
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 12s 349ms/step - accuracy: 0.8841 - loss: 0.2930 - val_accuracy: 0.6953 - val_loss: 2.2523
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 11s 347ms/step - accuracy: 0.9283 - loss: 0.1728 - val_accuracy: 0.6055 - val_loss: 4.2229
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 12s 351ms/step - accuracy: 0.9590 - loss: 0.1002 - val_accuracy: 0.5742 - val_loss: 2.8934
Epoch 8/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 20s 344ms/step - accuracy: 0.9289 - loss: 0.1586 - val_accu

In [78]:
# Get training metrics from history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

final_train_acc = acc[-1]
final_val_acc = val_acc[-1]

# NEW: Get the best validation accuracy during training (what EarlyStopping saved)
best_val_acc = max(val_acc)
best_epoch = val_acc.index(best_val_acc) + 1  # +1 because epochs start at 1

print('\nTraining Results:')
print(f'Final Training Accuracy: {final_train_acc*100:.2f}%')
print(f'Final Validation Accuracy: {final_val_acc*100:.2f}%')
print(f'\nBest Validation Accuracy: {best_val_acc*100:.2f}% (epoch {best_epoch})')

if best_val_acc > 0.83:
    print("\nModel mencapai target validation accuracy > 83%!")
else:
    print("\nModel belum mencapai target. Rekomendasi:")
    print("- Tingkatkan jumlah epoch (misal: 20-25)")


Training Results:
Final Training Accuracy: 98.83%
Final Validation Accuracy: 62.11%

Best Validation Accuracy: 81.64% (epoch 4)

Model belum mencapai target. Rekomendasi:
- Tingkatkan jumlah epoch (misal: 20-25)


**Save Model ke File**

In [79]:
model.save("model_05.h5")

In [80]:
from google.colab import files
files.download('model_05.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>